## GEDI L3 Example

### Purpose
Demonstrate how to sample the GEDI L3 rasters at generated ATL06-SR points

#### Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import sliderule
from sliderule import icesat2, raster

#### Initialize SlideRule Python Client

In [ ]:
icesat2.init("slideruleearth.io", verbose=True)

#### Make Processing Request to SlideRule
ATL06-SR request includes the `samples` parameter to specify that GEDI L3 Mean Elevation dataset should be sampled at each generated ATL06 elevation.

In [ ]:
asset = "icesat2"
resource = "ATL03_20220105023009_02111406_005_01.h5"
region = sliderule.toregion('grandmesa.geojson')
bbox = raster.poly2bbox(region["poly"])
parms = { "poly": region['poly'],
          "cnf": "atl03_high",
          "ats": 5.0,
          "cnt": 5,
          "len": 20.0,
          "res": 10.0,
          "maxi": 1,
          "samples": {"gedi": {"asset": "gedil3-elevation", "aoi_bbox": bbox}} 
}
gdf = icesat2.atl06p(parms, asset=asset, resources=[resource])

#### Display GeoDataFrame
Notice the columns that start with "gedi"

In [ ]:
gdf

#### Print Out File Directory
When a GeoDataFrame includes samples from rasters, each sample value has a file id that is used to look up the file name of the source raster for that value.

In [ ]:
gdf.attrs['file_directory']

In [ ]:
gdf["gedi.value"]

#### Plot the Different GEDI Values against the SlideRule ATL06-SR Values

In [ ]:
# Setup Plot
fig,ax = plt.subplots(num=None, figsize=(10, 8))
fig.set_facecolor('white')
fig.canvas.header_visible = False
ax.set_title("SlideRule vs. GEDI Elevations")
ax.set_xlabel('UTC')
ax.set_ylabel('height (m)')
legend_elements = []

# Plot SlideRule ATL06 Elevations
df = gdf[(gdf['rgt'] == 211) & (gdf['gt'] == 30) & (gdf['cycle'] == 14)]
sc1 = ax.scatter(df.index.values, df["h_mean"].values, c='red', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='red', lw=6, label='ATL06-SR'))

# Plot GEDI Elevations
sc2 = ax.scatter(df.index.values, df["gedi.value"].values, c='blue', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='blue', lw=6, label='GEDI'))

# Display Legend
lgd = ax.legend(handles=legend_elements, loc=3, frameon=True)
lgd.get_frame().set_alpha(1.0)
lgd.get_frame().set_edgecolor('white')

# Show Plot
plt.show()